In [1]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.1 MB/s eta 0:00:00


In [2]:
import torch
import timm
import gc
from torchvision.transforms.functional import resize, normalize, pil_to_tensor
from datasets import load_dataset

In [3]:
IMAGENET_DIR = '../input/imagenetmini-1000/imagenet-mini'

In [4]:
train_dataset = load_dataset('imagefolder', data_dir=IMAGENET_DIR + '/train')['train']

Resolving data files:   0%|          | 0/34745 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-9d80f747420ec4c2/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
train_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 34745
})

In [6]:
val_dataset = load_dataset('imagefolder', data_dir=IMAGENET_DIR + '/val')['train']

Resolving data files:   0%|          | 0/3923 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset image_folder downloaded and prepared to /root/.cache/huggingface/datasets/image_folder/default-0153cd34ff607c32/0.0.0/ee92df8e96c6907f3c851a987be3fd03d4b93b247e727b69a8e23ac94392a091. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
val_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 3923
})

In [8]:
torch.cuda.is_available()

True

In [9]:
device = torch.device('cuda')

In [10]:
def compute_beit():
    def compute_embedding(batch):
        tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
        tensors = [resize(tensor / 255., size=(224, 224)) for tensor in tensors]
        tensors = [normalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) for tensor in tensors]
        tensors = torch.stack(tensors)
        tensors = tensors.to(device=device)
        with torch.no_grad():
            batch['beit'] = beit(tensors).cpu().numpy()
        return batch

    beit = timm.create_model(
        'beit_large_patch16_224_in22k', pretrained=True, num_classes=0
    ).to(device=device)
    global train_dataset, val_dataset
    train_dataset = train_dataset.map(compute_embedding, batched=True)
    val_dataset = val_dataset.map(compute_embedding, batched=True)
    del beit
    torch.cuda.empty_cache()
    gc.collect()

In [11]:
def compute_vit():
    def compute_embedding(batch):
        tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
        tensors = [resize(tensor / 255., size=(224, 224)) for tensor in tensors]
        tensors = [normalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) for tensor in tensors]
        tensors = torch.stack(tensors)
        tensors = tensors.to(device=device)
        with torch.no_grad():
            batch['vit'] = vit(tensors).cpu().numpy()
        return batch

    vit = timm.create_model(
        'vit_base_patch32_224_in21k', pretrained=True, num_classes=0
    ).to(device=device)
    global train_dataset, val_dataset
    train_dataset = train_dataset.map(compute_embedding, batched=True)
    val_dataset = val_dataset.map(compute_embedding, batched=True)
    del vit
    torch.cuda.empty_cache()
    gc.collect()

In [12]:
def compute_efficientnet():
    def compute_embedding(batch):
        tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
        tensors = [resize(tensor / 255., size=(320, 320)) for tensor in tensors]
        tensors = [normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) for tensor in tensors]
        tensors = torch.stack(tensors)
        tensors = tensors.to(device=device)
        with torch.no_grad():
            batch['efficientnet'] = efficientnet(tensors).cpu().numpy()
        return batch

    efficientnet = timm.create_model(
        'efficientnet_b4', pretrained=True, num_classes=0
    ).to(device=device)
    global train_dataset, val_dataset
    train_dataset = train_dataset.map(compute_embedding, batched=True)
    val_dataset = val_dataset.map(compute_embedding, batched=True)
    del efficientnet
    torch.cuda.empty_cache()
    gc.collect()

In [13]:
def compute_convnext():
    def compute_embedding(batch):
        tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
        tensors = [resize(tensor / 255., size=(224, 224)) for tensor in tensors]
        tensors = [normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) for tensor in tensors]
        tensors = torch.stack(tensors)
        tensors = tensors.to(device=device)
        with torch.no_grad():
            batch['convnext'] = convnext(tensors).cpu().numpy()
        return batch

    convnext = timm.create_model(
        'convnext_large_in22k', pretrained=True, num_classes=0
    ).to(device=device)
    global train_dataset, val_dataset
    train_dataset = train_dataset.map(compute_embedding, batched=True)
    val_dataset = val_dataset.map(compute_embedding, batched=True)
    del convnext
    torch.cuda.empty_cache()
    gc.collect()


In [14]:
def compute_resnext():
    def compute_embedding(batch):
        tensors = [pil_to_tensor(x.convert('RGB')) for x in batch['image']]
        tensors = [resize(tensor / 255., size=(224, 224)) for tensor in tensors]
        tensors = [normalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) for tensor in tensors]
        tensors = torch.stack(tensors)
        tensors = tensors.to(device=device)
        with torch.no_grad():
            batch['resnext'] = resnext(tensors).cpu().numpy()
        return batch
    resnext = timm.create_model(
        'resnext101_32x8d', pretrained=True, num_classes=0
    ).to(device=device)
    global train_dataset, val_dataset
    train_dataset = train_dataset.map(compute_embedding, batched=True)
    val_dataset = val_dataset.map(compute_embedding, batched=True)
    del resnext
    torch.cuda.empty_cache()
    gc.collect()

In [15]:
if torch.cuda.is_available():
    compute_beit()
    #compute_vit()
    #compute_efficientnet()
    #compute_convnext()
    #compute_resnext()
    # save to disk
    train_dataset = train_dataset.remove_columns('image')
    train_dataset.save_to_disk('guie_preprocessed_train')
    val_dataset = val_dataset.remove_columns('image')
    val_dataset.save_to_disk('guie_preprocessed_val')

/opt/conda/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:2227.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://conversationhub.blob.core.windows.net/beit-share-public/beit/beit_large_patch16_224_pt22k_ft22k.pth" to /root/.cache/torch/hub/checkpoints/beit_large_patch16_224_pt22k_ft22k.pth


  0%|          | 0/35 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [16]:
train_dataset

Dataset({
    features: ['label', 'beit'],
    num_rows: 34745
})

In [17]:
val_dataset

Dataset({
    features: ['label', 'beit'],
    num_rows: 3923
})

In [18]:
!ls

__notebook__.ipynb  guie_preprocessed_train  guie_preprocessed_val
